In [ ]:
!pip install -q cohere pandas tqdm
import os
import cohere
import pandas as pd
from tqdm import tqdm
import re
import json
import time
import random

# Initialize Cohere client
co = cohere.Client('****************')  # Replace with your Cohere API key

# Load your CSV (adjust filename/path)
df = pd.read_csv("/content/DL_Energy_Patterns.csv")

# Preview relevant columns
df = df[['PostId', 'QUESTION', 'CHATGPT_ANSWER', 'SO_ANSWER']]
df.head()

# REMOVED: df = df.head(5) - Now analyzing all rows

prompt_template = """
Objective:
You are an expert software sustainability analyst evaluating code solutions for deep learning energy-efficiency. Your task is to analyze two solutions (Human Stack Overflow vs AI ChatGPT) using the established 8 energy-efficiency patterns with numerical scoring.

Task Description:
Evaluate both code solutions against the 8 core energy-efficiency patterns using a 1-5 scoring system. Provide a concise explanation highlighting key differences and evidence.

Scoring Framework (1-5 scale):
1: Poor - Pattern not implemented or severely deficient
2: Below Average - Basic implementation with significant issues
3: Average - Adequate implementation meeting basic requirements
4: Good - Solid implementation with clear energy benefits
5: Excellent - Exceptional implementation demonstrating best practices

Patterns and Associated Tactics:

1. PRE-TRAINED MODEL UTILIZATION
   - Measurable Tactics: Transfer Learning (T16), Knowledge Distillation (T17)
   - Evidence: Pre-trained model loading, fine-tuning, teacher-student architectures

2. CHECKPOINT MANAGEMENT
   - Measurable Tactics: Checkpoint Usage (T19)
   - Evidence: Model saving/loading strategies, resume training capabilities

3. MODEL OPTIMIZATION STRATEGIES
   - Measurable Tactics: Reduce Complexity (T8), Enhance Sparsity (T14), Energy-aware Pruning (T15)
   - Evidence: Simpler architectures, regularization, pruning implementations

4. QUANTIZATION TECHNIQUES
   - Measurable Tactics: Input Quantization (T4), Quantization-aware Training (T18)
   - Evidence: Precision reduction, quantization APIs, model optimization

5. EFFICIENT DATA HANDLING
   - Measurable Tactics: Sampling (T1), Remove Redundant Data (T2), Feature Reduction (T3), Minimize Data Referencing (T27)
   - Evidence: Data pipelines, sampling techniques, feature selection, generators

6. MEMORY MANAGEMENT
   - Measurable Tactics: Memory Constraints (T20), Computation Partitioning (T22)
   - Evidence: Device management, memory optimization, model partitioning

7. ALGORITHM & COMPUTATION OPTIMIZATION
   - Measurable Tactics: Energy-efficient Algorithms (T6), Lightweight Alternatives (T7), Dynamic Parameter Adaptation (T10), Built-in Library Functions (T11)
   - Evidence: Model architecture choices, adaptive learning, optimized operations

8. MODEL MAINTENANCE & ADAPTATION
   - Measurable Tactics: Graph Substitution (T13), Informed Adaptation (T28), Retrain When Needed (T29)
   - Evidence: Computation graph optimization, adaptive strategies

Evaluation Process:
1. Evaluate each of the 8 patterns independently for both solutions
2. Assign scores 1-5 based on implementation quality and evidence
3. Consider both explicit code implementations AND textual recommendations
4. For Human answers: Give credit for clear textual suggestions even without full code
5. For AI answers: Focus on actual code implementations and practical suggestions
6. Identify specific tactics (T1-T30) observed in the code or text
7. Calculate average scores for each solution
8. Determine winner based on higher average score
9. Provide concise analysis highlighting key differentiating factors

Important Scoring Guidelines:
- Code Evidence: Highest priority - explicit implementations get higher scores
- Textual Recommendations: Medium priority - clear suggestions without code get moderate scores
- Vague Mentions: Low priority - general statements without specifics get low scores
- Human Answers: Often contain valuable expert suggestions in natural language
- AI Answers: Typically provide complete code implementations

Tie Consideration:
- Primary determination is based on numerical scores (higher average wins)
- Objective tie condition: Only when score difference is exactly 0.0
- For minimal differences (≤0.125), carefully evaluate if solutions demonstrate truly complementary strengths that make them equally valuable
- Consider tie ONLY when both solutions excel in different but equally important aspects that balance each other
- Avoid automatic ties for small differences - the higher score should generally win unless there are compelling complementary factors

Reporting Instructions:
Return ONLY a JSON object with this exact structure:
{{
  "human_pattern_1_score": 1-5,
  "human_pattern_2_score": 1-5,
  "human_pattern_3_score": 1-5,
  "human_pattern_4_score": 1-5,
  "human_pattern_5_score": 1-5,
  "human_pattern_6_score": 1-5,
  "human_pattern_7_score": 1-5,
  "human_pattern_8_score": 1-5,

  "ai_pattern_1_score": 1-5,
  "ai_pattern_2_score": 1-5,
  "ai_pattern_3_score": 1-5,
  "ai_pattern_4_score": 1-5,
  "ai_pattern_5_score": 1-5,
  "ai_pattern_6_score": 1-5,
  "ai_pattern_7_score": 1-5,
  "ai_pattern_8_score": 1-5,

  "human_average_score": "calculated average",
  "ai_average_score": "calculated average",
  "score_difference": "difference in averages",
  "winner": "Human" or "AI" or "Tie",
  "analysis_explanation": "Brief explanation for your analysis, highlighting key differentiating factors in human and AI solutions, citing relevant code snippets. In case of tie, explain compelling complementary strengths that justify equal evaluation despite numerical differences."
}}

Examples:
Example output of a tie:
{{
  "human_pattern_1_score": 5,
  "human_pattern_2_score": 3,
  "human_pattern_3_score": 4,
  "human_pattern_4_score": 2,
  "human_pattern_5_score": 4,
  "human_pattern_6_score": 3,
  "human_pattern_7_score": 4,
  "human_pattern_8_score": 3,

  "ai_pattern_1_score": 3,
  "ai_pattern_2_score": 4,
  "ai_pattern_3_score": 3,
  "ai_pattern_4_score": 5,
  "ai_pattern_5_score": 3,
  "ai_pattern_6_score": 4,
  "ai_pattern_7_score": 3,
  "ai_pattern_8_score": 5,

  "human_average_score": 3.5,
  "ai_average_score": 3.625,
  "score_difference": 0.125,
  "winner": "Tie",
  "analysis_explanation": "Despite AI's slight numerical advantage (0.125), both solutions demonstrate truly complementary and equally valuable approaches. Human excels in Pre-trained Model Utilization (P1: score 5) with expert-level transfer learning strategies and practical fine-tuning guidance that significantly reduces training energy. AI counters with exceptional Quantization Techniques (P4: score 5) and Model Maintenance (P8: score 5) through advanced optimization implementations. The Human solution provides deep domain expertise that could save substantial computational resources in model development, while AI offers robust technical implementations for runtime efficiency. Given that both approaches address fundamentally different but equally critical aspects of the energy-efficiency pipeline, this evaluation results in a tie."
}}

Example output with clear winner:
{{
  "human_pattern_1_score": 4,
  "human_pattern_2_score": 3,
  "human_pattern_3_score": 2,
  "human_pattern_4_score": 1,
  "human_pattern_5_score": 4,
  "human_pattern_6_score": 3,
  "human_pattern_7_score": 3,
  "human_pattern_8_score": 2,

  "ai_pattern_1_score": 3,
  "ai_pattern_2_score": 4,
  "ai_pattern_3_score": 3,
  "ai_pattern_4_score": 4,
  "ai_pattern_5_score": 3,
  "ai_pattern_6_score": 4,
  "ai_pattern_7_score": 4,
  "ai_pattern_8_score": 3,

  "human_average_score": 2.75,
  "ai_average_score": 3.5,
  "score_difference": 0.75,
  "winner": "AI",
  "analysis_explanation": "Both solutions effectively address the core `sess.run()` bottleneck by fetching entire weight matrices at once, improving Memory Management (P6) by reducing overhead and data transfers (Human: `h1_val = sess.run(weights['h1'])`, AI: `weight_values = sess.run(tensor)`). The Human solution provides a solid Algorithm & Computation Optimization (P7) by fixing the data extraction, but still uses manual loops for writing. The AI solution excels here by implementing `np.savetxt(fp, weight_values)`, leveraging highly optimized built-in libraries for superior computational efficiency and streamlined I/O. AI's more complete and robust approach, including support for multiple layers, provides a clearer best practice for energy-efficient weight serialization."
}}

---
Now evaluate the following:

Question:
{question}

Human (Stack Overflow) Answer:
{human_answer}

AI (ChatGPT) Answer:
{ai_answer}
"""

# Update the pattern_columns list for the simplified output structure
pattern_columns = []

# Add all 8 pattern scores for human and AI
for i in range(1, 9):
    pattern_columns.append(f'human_pattern_{i}_score')
for i in range(1, 9):
    pattern_columns.append(f'ai_pattern_{i}_score')

# Add summary columns
pattern_columns.extend([
    'human_average_score', 'ai_average_score', 'score_difference',
    'winner', 'analysis_explanation'
])

# Initialize empty columns
for col in pattern_columns:
    df[col] = None

def parse_json_response(response_text):
    """Extract JSON from response text and parse it with better error handling"""
    try:
        # Remove markdown code blocks if present
        cleaned_text = response_text.strip()

        # Remove ```json and ``` markers
        if cleaned_text.startswith('```json'):
            cleaned_text = cleaned_text[7:]  # Remove ```json
        if cleaned_text.startswith('```'):
            cleaned_text = cleaned_text[3:]  # Remove ```
        if cleaned_text.endswith('```'):
            cleaned_text = cleaned_text[:-3]  # Remove ```

        cleaned_text = cleaned_text.strip()

        # Try to parse the cleaned text
        return json.loads(cleaned_text)

    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        print(f"Trying alternative parsing for response: {response_text[:200]}...")

        # Alternative approach: try to find JSON object more robustly
        try:
            # Look for the first { and last }
            start_idx = response_text.find('{')
            end_idx = response_text.rfind('}') + 1

            if start_idx != -1 and end_idx != 0:
                json_str = response_text[start_idx:end_idx]
                return json.loads(json_str)
        except Exception as e2:
            print(f"Alternative parsing also failed: {e2}")

    except Exception as e:
        print(f"Unexpected error parsing JSON: {e}")

    return None

# Define checkpoint interval and rate limiting
CHECKPOINT_INTERVAL = 20
base_output_path = "/content/cohere_final"

# Rate limiting parameters
REQUESTS_PER_MINUTE = 8  # Conservative limit (trial allows 10, using 8 for safety)
DELAY_BETWEEN_REQUESTS = 60 / REQUESTS_PER_MINUTE  # Calculate delay in seconds

# Create a copy to track processed rows
processed_indices = set()
last_request_time = 0

for idx, row in tqdm(df.iterrows(), total=len(df)):
    # Skip if already processed (in case of restart)
    if pd.notna(df.at[idx, 'human_pattern_1_score']):
        processed_indices.add(idx)
        continue

    question = str(row['QUESTION'])
    human_answer = str(row['SO_ANSWER'])
    ai_answer = str(row['CHATGPT_ANSWER'])

    prompt = prompt_template.format(
        question=question,
        human_answer=human_answer,
        ai_answer=ai_answer
    )

    # Rate limiting: ensure we don't exceed API limits
    current_time = time.time()
    time_since_last_request = current_time - last_request_time

    if time_since_last_request < DELAY_BETWEEN_REQUESTS:
        sleep_time = DELAY_BETWEEN_REQUESTS - time_since_last_request
        print(f"⏳ Rate limiting: waiting {sleep_time:.2f} seconds...")
        time.sleep(sleep_time)

    retry_count = 0
    max_retries = 3

    while retry_count < max_retries:
        try:
            last_request_time = time.time()

            # Use Cohere's Chat API with more specific instructions
            response = co.chat(
                model='command-a-03-2025',
                message=prompt,
                max_tokens=2000,  # Increased for longer JSON responses
                temperature=0.1,
                preamble="You must respond with ONLY valid JSON format, no additional text, explanations, or markdown formatting."
            )

            json_data = parse_json_response(response.text)

            if json_data:
                # Update the dataframe with the parsed JSON data
                for key, value in json_data.items():
                    if key in df.columns:
                        df.at[idx, key] = value
                processed_indices.add(idx)
                print(f"✅ Successfully processed row {idx}")
                break  # Success, break out of retry loop
            else:
                print(f"❌ Failed to parse JSON for row {idx}")
                print(f"Raw response: {response.text}")
                break  # Parsing failed, don't retry

        except Exception as e:
            retry_count += 1
            error_str = str(e)

            if "429" in error_str or "rate limit" in error_str.lower():
                if retry_count < max_retries:
                    # Exponential backoff for rate limits
                    wait_time = (2 ** retry_count) + random.uniform(0, 1)
                    print(f"🚨 Rate limit hit (attempt {retry_count}/{max_retries}). Waiting {wait_time:.2f} seconds...")
                    time.sleep(wait_time)
                else:
                    print(f"🚨 Rate limit persists after {max_retries} attempts. Saving progress and pausing...")
                    # Save current progress before exiting
                    batch_number = (idx + 1) // CHECKPOINT_INTERVAL
                    if batch_number == 0:
                        batch_number = 1
                    batch_file = f"{base_output_path}_batch_{batch_number}.csv"
                    df.to_csv(batch_file, index=False)
                    print(f"💾 Progress saved to {batch_file}. Please wait a minute and resume.")
                    break
            else:
                print(f"🚨 Error on row {idx}: {e}")
                break  # Other errors, don't retry

    # Save checkpoint every CHECKPOINT_INTERVAL rows
    if (idx + 1) % CHECKPOINT_INTERVAL == 0:
        batch_number = (idx + 1) // CHECKPOINT_INTERVAL
        batch_file = f"{base_output_path}_batch_{batch_number}.csv"

        # Get the current batch (rows 0 to idx)
        current_batch = df.iloc[:(idx + 1)]

        print(f"\n💾 Saving batch {batch_number} (rows 0-{idx}) to {batch_file}...")
        current_batch.to_csv(batch_file, index=False)
        print(f"✅ Batch {batch_number} saved! Contains {len(current_batch)} rows.")

        # Additional pause after each batch to be extra safe
        print("😴 Taking a short break after batch completion...")
        time.sleep(2)

# Final saves after all rows are processed
print("\n🎯 Final save...")

# Save the final combined CSV with all rows
final_combined_file = f"{base_output_path}_complete.csv"
df.to_csv(final_combined_file, index=False)
print(f"✅ Complete dataset saved to {final_combined_file}!")

# Also save individual batches for the remaining rows
total_rows = len(df)
if total_rows % CHECKPOINT_INTERVAL != 0:
    final_batch_number = (total_rows // CHECKPOINT_INTERVAL) + 1
    final_batch_file = f"{base_output_path}_batch_{final_batch_number}.csv"
    df.to_csv(final_batch_file, index=False)
    print(f"✅ Final batch {final_batch_number} saved to {final_batch_file}!")

print(f"🎉 Analysis complete! Processed {len(processed_indices)} rows total.")

# Display the structured results
print("\n📊 Structured Results:")
print(df[['PostId', 'QUESTION'] + pattern_columns].head())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 56.0 MB/s eta 0:00:00


  1%|          | 1/190 [00:05<18:25,  5.85s/it]

✅ Successfully processed row 0
⏳ Rate limiting: waiting 1.65 seconds...


  1%|          | 2/190 [00:12<20:16,  6.47s/it]

✅ Successfully processed row 1
⏳ Rate limiting: waiting 2.24 seconds...


...

✅ Successfully processed row 187
⏳ Rate limiting: waiting 1.42 seconds...


 99%|█████████▉| 189/190 [23:49<00:07,  7.36s/it]

✅ Successfully processed row 188
⏳ Rate limiting: waiting 2.03 seconds...


100%|██████████| 190/190 [23:57<00:00,  7.56s/it]

✅ Successfully processed row 189

🎯 Final save...
✅ Complete dataset saved to /content/cohere_final_complete.csv!
✅ Final batch 10 saved to /content/cohere_final_batch_10.csv!
🎉 Analysis complete! Processed 190 rows total.

📊 Structured Results:
     PostId                                           QUESTION  \
0  37511148  How to quickly print Tensorflow results to TXT...   
1  54261772  Tensorflow Lite toco --mean_values --std_value...   
2  63178757  "Processus stopped "during prediction after mo...   
3  44311820  Tensorflow tanh with quantized values I am exp...   
4  43505001  Computing bincount (or spectrum) of matrix ove...   

  human_pattern_1_score human_pattern_2_score human_pattern_3_score  \
0                     2                     1                     1   
1                     2                     1                     1   
2                     1                     1                     1   
3                     1                     1                     1   
4 